In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nnfs
import math
import random

from nnfs.datasets import spiral_data
from nnfs.datasets import vertical_data

from layers import Dense, Dropout
from activations import ReLU
from activations import SoftMax
from losses import CategoricalCrossEntropy, Softmax_CategoricalCrossentropy
from optimizers import SGD, AdaGrad, RMSProp, Adam

In [3]:
nnfs.init()

## CH 16: Binary Logistic Regression

In [10]:
# Dataset
X, y = spiral_data(samples=100, classes=3)

# First Layer
dense1 = Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = ReLU()
dropout1 = Dropout(0.1)

# Second Layer
dense2 = Dense(64, 3)

# Categorical-CrossEntropy with Activation
loss_activation = Softmax_CategoricalCrossentropy()

# Optimizer
optimizer = Adam(decay=5e-5, learning_rate=0.05)

for epoch in range(10001):
    # Forward Pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dropout1.forward(activation1.output)

    dense2.forward(dropout1.output)
    
    # Loss Computation
    # Data
    data_loss = loss_activation.forward(dense2.output, y)
    
    # Regularization termr
    regularization_loss = loss_activation.regularization_loss(dense1) + loss_activation.regularization_loss(dense2)
    
    # Total
    loss = data_loss + regularization_loss
    
    # Accuracy
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) ==2:
        y = np.argmax(y, axis=1)

    acc = np.mean(predictions==y)
    
    if not epoch % 1000:
        print(f'epoch: {epoch}, ' +
              f'acc: {acc:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'data_loss: {data_loss:.3f} ' +
              f'reg_loss: {regularization_loss:.3f} ' +
              f'lr: {optimizer.current_learning_rate} ')

    # Backward Pass
    loss_activation.backward(loss_activation.output, y)

    dense2.backward(loss_activation.dinputs)
    
    dropout1.backward(dense2.dinputs)
    activation1.backward(dropout1.dinputs)
    dense1.backward(activation1.dinputs)

    # Optimize
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.370, loss: 1.099, data_loss: 1.099 reg_loss: 0.000 lr: 0.05 
epoch: 1000, acc: 0.693, loss: 0.767, data_loss: 0.722 reg_loss: 0.045 lr: 0.04762131530072861 
epoch: 2000, acc: 0.647, loss: 0.771, data_loss: 0.727 reg_loss: 0.044 lr: 0.045456611664166556 
epoch: 3000, acc: 0.710, loss: 0.658, data_loss: 0.612 reg_loss: 0.046 lr: 0.043480151310926564 
epoch: 4000, acc: 0.700, loss: 0.696, data_loss: 0.649 reg_loss: 0.047 lr: 0.04166840285011875 
epoch: 5000, acc: 0.747, loss: 0.640, data_loss: 0.593 reg_loss: 0.048 lr: 0.04000160006400256 
epoch: 6000, acc: 0.730, loss: 0.649, data_loss: 0.602 reg_loss: 0.047 lr: 0.03846301780837725 
epoch: 7000, acc: 0.720, loss: 0.661, data_loss: 0.614 reg_loss: 0.046 lr: 0.03703840882995667 
epoch: 8000, acc: 0.723, loss: 0.662, data_loss: 0.616 reg_loss: 0.046 lr: 0.03571556127004536 
epoch: 9000, acc: 0.723, loss: 0.737, data_loss: 0.691 reg_loss: 0.047 lr: 0.034483947722335255 
epoch: 10000, acc: 0.723, loss: 0.668, data_loss: 0.622

In [11]:
# Create test set
X_test, y_test = spiral_data(samples=100, classes=3)

# Evaluate the model on test set
# Forward pass

dense1.forward(X_test)
activation1.forward(dense1.output)

dense2.forward(activation1.output)

loss = loss_activation.forward(dense2.output, y_test)

predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) ==2:
    y_test = np.argmax(y_test, axis=1)

acc = np.mean(predictions==y_test)

print(f'Validation acc: {acc:.3f}, loss: {loss:.3f}')

Validation acc: 0.717, loss: 0.691
